In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
import pandas as pd
import random
import seaborn as sns
import sklearn
import time
import torch
#import umap

# Non-Ipython imports should be sorted by name
from IPython import get_ipython
from IPython.core.debugger import set_trace



In [3]:
#export
markers = ['.', 'v', '^', '>', '<', '8', 's', 'p', 'P', '*', 'h', 'H', 'X', 'D', 'd']

In [4]:
#export
def groupify(l,t):
    l = l.type(torch.FloatTensor)
    t  = torch.cat((l.unsqueeze(dim=1),t), dim=1)
    l  = set(sorted(l.tolist()))
    gs = {}
    for c in l:
        g = []
        for i in range(t.shape[0]):
            if t[i][0] == c: g.append(t[i])
        gs.update({c:torch.stack(g)})
    return gs

def groupifyn(l,d):
    '''
    Make groups of label-data dict for numpy arrays
    l : numpy list of items. Will be uniquely identified for elements.
    d : numpy data of the same length as l.
    '''
    ls  = set(sorted(l.tolist()))
    if len(l.shape) ==1 : l = np.expand_dims(l, axis=1)
    ld = np.concatenate((l, d), axis=1)
    gs = {}
    for c in ls:
        g = []
        for i in range(ld.shape[0]):
            if ld[i][0] == c: g.append(ld[i])
        gs.update({c:np.stack(g)})
    return gs

In [5]:
#export
def PCA(x, k=2):
    xm = (torch.mean(x)).expand_as(x)
    x = x - xm
    u,s,v = torch.svd(torch.t(x))
    return torch.mm(x, u[:,:k])

In [6]:
#export
def plot_group_scatter_t(labels, groups, title, figsize=[10,10]):
    '''
    scatter plot with tensor based groups
    '''
    colors = cm.nipy_spectral(np.linspace(0, 1, len(labels))) # cm.rainbow has limited colors, try cm.nipy_spectral or cm.gist_ncar
    fig = plt.figure(figsize=figsize)
    k = list(groups)[0].shape[1] -1
    if   k == 2: # 2D plot
        ax = fig.add_subplot(1, 1, 1)
        for lbl, grp, clr in zip(labels, groups, colors):
            ax.scatter(grp.narrow(1,1,1).numpy(), grp.narrow(1,2,1).numpy(), alpha=0.8, color=clr, marker=random.choice(markers), edgecolors='none', s=40, label=lbl)
    elif k == 3: # 3D plot
        ax = plt.axes(projection="3d")
        for lbl, grp, clr in zip(labels, groups, colors):
            ax.scatter3D(grp.narrow(1,1,1).numpy(), grp.narrow(1,2,1).numpy(), grp.narrow(1,3,1).numpy(), color=clr, marker=random.choice(markers), s=40 ,alpha=1, edgecolor='grey', label=lbl)        
    plt.title(title)
    plt.show()
    return fig

def plot_group_scatter_n(labels, groups, title, figsize=[10,10]):
    colors = cm.nipy_spectral(np.linspace(0, 1, len(labels)))
    fig = plt.figure(figsize=figsize)
    k = list(groups)[0].shape[1] -1
    if   k == 2: # 2D plot 
        ax = fig.add_subplot(1, 1, 1)
        for lbl, grp, clr in zip(labels, groups, colors):
            ax.scatter(grp[:,1:2], grp[:,2:3], alpha=0.8, color=clr, marker=random.choice(markers), edgecolors='none', s=40, label=lbl)
    elif k == 3: # 3D plot
        ax = plt.axes(projection="3d")
        for lbl, grp, clr in zip(labels, groups, colors):
            ax.scatter3D(grp[:,1:2], grp[:,2:3], grp[:,3:4], color=clr, marker=random.choice(markers), s=40, alpha=1, edgecolor='grey', label=lbl)        
    plt.title(title)
    plt.show()
    return fig

In [7]:
#export
def plot_pca(l, emb, k=3, figsize=[10,10]):
    emb_pca = PCA(emb, k=k)
    groups = groupify(l, emb_pca)
    return plot_group_scatter_t(groups.keys(), groups.values(), f'PCA for C=({len(groups.keys())})', figsize=figsize)

In [8]:
! ../notebook2script.py plot_utils.ipynb

Converted plot_utils.ipynb to exp/plot_utils.py
